In [2]:
# !pip install torch-geometric
# !pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.8 MB/s eta 0:00:0000:0100:01


In [3]:
# import argparse
# import networkx as nx
# import numpy as np
# import matplotlib
# matplotlib.use('Agg')
# import matplotlib.pyplot as plt

# import torch
# import torch.nn.functional as F
# from torch_geometric.nn import GraphSAGE  # or SAGEConv + custom
# from torch_geometric.data import Data
# from torch_geometric.utils import negative_sampling, structured_negative_sampling
# from torch_geometric.loader import NeighborLoader

# from sentence_transformers import SentenceTransformer
# from sklearn.model_selection import train_test_split

# import os
# from tqdm import tqdm
# import re
# from rapidfuzz import process, fuzz

2025-05-05 10:12:58.041660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746439978.248440      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746439978.308625      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# # GRAPH BUILDER CODE
# def extract_brace_content(text, start_idx):
#     """Extract content inside balanced braces starting from the given index."""
#     assert text[start_idx] == '{'
#     depth = 0
#     i = start_idx
#     while i < len(text):
#         if text[i] == '{':
#             depth += 1
#         elif text[i] == '}':
#             depth -= 1
#             if depth == 0:
#                 return text[start_idx+1:i], i + 1
#         i += 1
#     return "", i

# def parse_bbl(bbl_path='t.bbl'):
#     """Extract reference titles from .bbl file supporting multiple formats."""
#     try:
#         with open(bbl_path, 'r', encoding='utf8') as f:
#             text = f.read()
#     except UnicodeDecodeError:
#         with open(bbl_path, 'r', encoding='latin-1') as f:
#             text = f.read()

#     titles = []

#     # 1. BibLaTeX \entry{...} blocks
#     entry_blocks = re.findall(r'\\entry\{.*?\}\{.*?\}\{.*?\}(.*?)\\endentry', text, flags=re.DOTALL)
#     for block in entry_blocks:
#         idx = block.find(r'\field{title}')
#         if idx != -1:
#             brace_start = block.find('{', idx + len(r'\field{title}'))
#             if brace_start != -1:
#                 raw_title, _ = extract_brace_content(block, brace_start)
#                 title = clean_title(raw_title)
#                 titles.append(title)

#     # 2. \bib{key}{type}{...} blocks
#     bib_blocks = re.findall(r'\\bib\{.*?\}\{.*?\}\{(.*?)\n\}', text, flags=re.DOTALL)
#     for block in bib_blocks:
#         match = re.search(r'title\s*=\s*\{(.*?)\}', block, flags=re.DOTALL)
#         if match:
#             title = clean_title(match.group(1))
#             titles.append(title)

#     # 3. \bibitem blocks
#     bibitems = re.split(r"\\bibitem(?:\[[^\]]*\]){0,3}\{.*?\}", text)[1:]
#     for block in bibitems:
#         title = None

#         # (a) \showarticletitle{...}
#         match = re.search(r'\\showarticletitle\s*\{(.*?)\}', block, flags=re.DOTALL)
#         if match:
#             title = clean_title(match.group(1))
#         else:
#             # (b) \newblock ...
#             parts = re.split(r"\\newblock", block)
#             if len(parts) >= 2:
#                 title = clean_title(parts[1])
#             else:
#                 # (c) ``...'' quoted title
#                 quote_match = re.search(r"``(.*?)''", block, flags=re.DOTALL)
#                 if quote_match:
#                     title = clean_title(quote_match.group(1))
#                 else:
#                     # (d) : {Title}
#                     colon_title = re.search(r':[ \n]*\{(.*?)\}', block, flags=re.DOTALL)
#                     if colon_title:
#                         title = clean_title(colon_title.group(1))

#         if title:
#             titles.append(title)

#     # 4. Handle cases like \url{...}
#     urls = re.findall(r'\\url\{(.*?)\}', text)
#     for url in urls:
#         # Optionally, save URLs if needed
#         pass
#     # if len(titles)==0:
#     #     print(bbl_path)
#     return titles

# def parse_bib(bib_file_path):
#     try:
#         with open(bib_file_path, 'r', encoding='utf8') as f:
#             bib_string = f.read()
#     except UnicodeDecodeError:
#         with open(bib_file_path, 'r', encoding='latin-1') as f:
#             bib_string = f.read()

#     # pattern = r'title\s*=\s*(?:\{(.*?)\}|"(.*?)")'
#     # matches = re.findall(pattern, bib_string, flags=re.IGNORECASE | re.DOTALL)

#     # cleaned_titles = []
#     # for brace_match, quote_match in matches:
#     #     title = brace_match if brace_match else quote_match

#     #     title = title.strip()
#     #     title = re.sub(r"^[\{\}]+|[\{\}]+$", "", title)
#     #     cleaned_titles.append(title.strip())

#     # return cleaned_titles
#     pattern = r'(?:^|\n|,\s*)title\s*=\s*(?:\{|")(.*?)(?:\}|")'
#     matches = re.findall(pattern, bib_string, flags=re.IGNORECASE | re.DOTALL)

#     cleaned_titles = []
#     for title in matches:
#         # The regex now directly captures the content within braces/quotes in group 1
#         title = title.strip()
#         # Apply the brace remover just in case of nested braces (less common now)
#         # and final strip.
#         cleaned_titles.append(re.sub(r"^[\{\}]+|[\{\}]+$", "", title).strip())

#     return cleaned_titles

# def clean_title(raw):
#     """Cleans LaTeX formatting from title string."""
#     raw = raw.replace('\n', ' ')
#     raw = re.sub(r'\s+', ' ', raw)
#     raw = re.sub(r'\{\\[a-zA-Z]+\s+(.*?)\}', r'\1', raw)   # {\em foo} → foo
#     raw = re.sub(r'\\[a-zA-Z]+\s*', '', raw)               # \em, \url, etc
#     raw = re.sub(r'[{}]', '', raw)                         # remove braces
#     return raw.strip(' .')



# def build_title_map(data_dir):
#     """Map paper_id -> paper title from title.txt."""
#     title_map = {}
#     cnt = 0
#     print("toal = ", len(os.listdir(data_dir)))
#     for paper in os.listdir(data_dir):
#         cnt += 1
#         if cnt %1000 == 0:
#             print(cnt)
#         pap_dir = os.path.join(data_dir, paper)
#         tfile = os.path.join(pap_dir, 'title.txt')
#         if os.path.isdir(pap_dir) and os.path.isfile(tfile):
#             with open(tfile, 'r', encoding='utf8') as f:
#                 title_map[paper] = f.read().strip()
#     return title_map

# def build_references(data_dir):
#     """Map paper_id -> list of reference titles."""
#     refs = {}
#     for paper in os.listdir(data_dir):
#         pap_dir = os.path.join(data_dir, paper)
#         if not os.path.isdir(pap_dir):
#             continue
#         # find .bbl
#         found_bbl = False
#         for fname in os.listdir(pap_dir):
#             if fname.endswith('.bbl'):
#                 found_bbl = True
#                 bbl_path = os.path.join(pap_dir, fname)
#                 refs[paper] = parse_bbl(bbl_path)
#                 break
#         if not found_bbl:
#             for fname in os.listdir(pap_dir):
#                 if fname.endswith('.bib'):
#                     bib_path = os.path.join(pap_dir, fname)
#                     refs[paper] = parse_bib(bib_path)
#                     break
#     return refs

# def match_refs_to_papers(refs, title_map, threshold=80):
#     """Fuzzy match reference titles to known paper titles."""
#     graph = nx.DiGraph()
#     # add nodes
#     for pid in title_map:
#         graph.add_node(pid)
#     titles = list(title_map.values())
#     pid_by_title = {v: k for k, v in title_map.items()}

#     for src, rlist in tqdm(refs.items(), desc='Matching refs'):
#         for ref in rlist:
#             match, score, _ = process.extractOne(ref, titles, scorer=fuzz.token_set_ratio)
#             if score >= threshold:
#                 tgt = pid_by_title.get(match)
#                 if tgt:
#                     graph.add_edge(src, tgt)
#     return graph

In [5]:
# # INVOCATION OF GRAPH BUILDER
# def generate_graphml():  
#     data_dir = "/kaggle/input/papers/dataset_papers"
#     out = "citation_graph.graphml"
    
#     title_map = build_title_map(data_dir)
#     print("1")
#     refs = build_references(data_dir)
#     print("2")
#     G = match_refs_to_papers(refs, title_map)
#     print("3")
#     nx.write_graphml(G, out)
#     print(G.number_of_edges(), G.number_of_nodes())
#     # print(f"Graph written to {args.out} with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

In [6]:
# generate_graphml()

toal =  6545
1000
2000
3000
4000
5000
6000
1
2


Matching refs: 100%|██████████| 6545/6545 [1:05:45<00:00,  1.66it/s]


3
28686 6545


In [7]:

# # 1. Prepare text embeddings
# sbert = SentenceTransformer('all-MiniLM-L6-v2')

# class LinkPredictor(torch.nn.Module):
#     def __init__(self, in_dim=384, hidden=128, num_layers=2):
#         super().__init__()
#         # Initialize GraphSAGE with specified number of layers
#         self.sage = GraphSAGE(in_channels=in_dim, hidden_channels=hidden, num_layers=num_layers)
#         # self.lin = torch.nn.Linear(hidden*2, 1)

#     def forward(self, x, edge_index, pos_edge_label_index, neg_edge_label_index):
#         h = self.sage(x, edge_index)
#         # # positive
#         # pos_h = torch.cat([h[pos_edge_label_index[0]], h[pos_edge_label_index[1]]], dim=1)
#         # neg_h = torch.cat([h[neg_edge_label_index[0]], h[neg_edge_label_index[1]]], dim=1)
#         # pos_score = self.lin(pos_h).view(-1)
#         # neg_score = self.lin(neg_h).view(-1)
#         # return pos_score, neg_score
#             # Dot product for positive edges
#         pos_score = (h[pos_edge_label_index[0]] * h[pos_edge_label_index[1]]).sum(dim=1)
        
#         # Dot product for negative edges
#         neg_score = (h[neg_edge_label_index[0]] * h[neg_edge_label_index[1]]).sum(dim=1)
        
#         return pos_score, neg_score


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
# def load_graph1(graph_path):
#     Gnx = nx.read_graphml(graph_path)
#     mapping = {nid: i for i, nid in enumerate(Gnx.nodes())}
#     # build edge_index
#     edges = [[mapping[u], mapping[v]] for u, v in Gnx.edges()]
#     edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
#     return edge_index, mapping


# def load_text_embeddings(data_dir, mapping, device):
#     sbert = SentenceTransformer('all-MiniLM-L6-v2', device=device)
#     x = torch.zeros((len(mapping), 384), dtype=torch.float, device=device)
#     for nid, idx in tqdm(mapping.items(), desc='Embedding texts'):
#         txt = ''
#         for fname in ('title.txt', 'abstract.txt'):
#             path = os.path.join(data_dir, nid, fname)
#             if os.path.isfile(path):
#                 txt += open(path, 'r', encoding='utf8').read().strip() + ' '
#         if txt:
#             emb = sbert.encode(txt, convert_to_tensor=True, show_progress_bar=False)
#             x[idx] = emb
#     x = F.normalize(x, p=2, dim=1)
#     return x


# def train(args, x):
#     device = torch.device(args.device)
#     print(device)
#     edge_index, mapping = load_graph1(args.graph)
#     edge_index = edge_index.to(device)

#     # split pos edges into train/val
#     num_edges = edge_index.size(1)
#     perm = torch.randperm(num_edges)
#     val_cnt = int(0.1 * num_edges)
#     val_idx = perm[:val_cnt]
#     train_idx = perm[val_cnt:]
#     pos_train = edge_index[:, train_idx]
#     pos_val = edge_index[:, val_idx]

#     model = LinkPredictor(in_dim=x.size(1), hidden=128, num_layers=2).to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

#     for epoch in range(1, args.epochs+1):
#         print(epoch)
#         model.train()
#         optimizer.zero_grad()
#         neg_train = negative_sampling(edge_index, num_neg_samples=pos_train.size(1), method='sparse')
#         pos_out, neg_out = model(x, edge_index, pos_train, neg_train)
#         loss = (
#             F.binary_cross_entropy_with_logits(pos_out, torch.ones_like(pos_out)) +
#             F.binary_cross_entropy_with_logits(neg_out, torch.zeros_like(neg_out))
#         ) / 2
#         loss.backward()
#         optimizer.step()

#         model.eval()
#         with torch.no_grad():
#             neg_val = negative_sampling(edge_index, num_neg_samples=pos_val.size(1), method='sparse')
#             pos_v, neg_v = model(x, edge_index, pos_val, neg_val)
#             val_loss = (
#                 F.binary_cross_entropy_with_logits(pos_v, torch.ones_like(pos_v)) +
#                 F.binary_cross_entropy_with_logits(neg_v, torch.zeros_like(neg_v))
#             ) / 2
#         print(f'Epoch {epoch}/{args.epochs} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}')

#     # os.makedirs(os.path.dirname(args.out_model), exist_ok=True)
#     # torch.save(model.state_dict(), args.out_model)
#     # save final node embeddings
#     emb = model.sage(x, edge_index).cpu()
#     print(emb.shape)
#     with torch.no_grad():
#         per_node_std = emb.std(dim=1)
#         per_node_mean = emb.mean(dim=1)
#         print(per_node_std[:10])
#         print(per_node_mean[:10])
#         print(f"Mean of per-node std: {per_node_std.mean().item():.6f}")
#         print(f"Mean of per-node mean: {per_node_mean.mean().item():.6f}")


#     # torch.save(emb, os.path.join(os.path.dirname(args.out_model), 'node_embeddings.pt'))
#     print('Training complete. Model and embeddings saved.')
#     torch.save(model, '/kaggle/working/model.pth')
#     return model.state_dict(), emb


In [23]:
# class Config:
#     graph = '/kaggle/input/final-graph/final_citations_graph.graphml'
#     # graph = '/kaggle/working/citation_graph.graphml'
#     data_dir = '/kaggle/input/papers/dataset_papers'          # Replace with actual path
#     epochs = 200
#     lr = 2e-3
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     out_model = 'linkpred.pt'  # You can save to /kaggle/working/

# def wrap_train(text_emb_path = '/kaggle/input/text-embeddings/text_embeddings.pt'):
#     args = Config()
    
#     _, mapping = load_graph1(args.graph)
#     device = torch.device(args.device)
    
#     # x = load_text_embeddings(args.data_dir, mapping, device)
#     # torch.save(x.cpu(), text_emb_path)

#     x = torch.load(text_emb_path, map_location=device)
    
#     state_dict, emb = train(args, x)
#     torch.save(state_dict, '/kaggle/working/'+args.out_model)
#     torch.save(emb, '/kaggle/working/'+'node_embeddings.pt')

In [24]:
# wrap_train()

/tmp/ipykernel_31/1672017970.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x = torch.load(text_emb_path, map_location=device)


cuda
1
Epoch 1/200 | Train Loss: 0.7088 | Val Loss: 0.6962
2
Epoch 2/200 | Train Loss: 0.6962 | Val Loss: 0.6839
3
Epoch 3/200 | Train Loss: 0.6837 | Val Loss: 0.6741
4
Epoch 4/200 | Train Loss: 0.6708 | Val Loss: 0.6785
5
Epoch 5/200 | Train Loss: 0.6748 | Val Loss: 0.6600
6
Epoch 6/200 | Train Loss: 0.6575 | Val Loss: 0.6553
7
Epoch 7/200 | Train Loss: 0.6567 | Val Loss: 0.6504
8
Epoch 8/200 | Train Loss: 0.6517 | Val Loss: 0.6424
9
Epoch 9/200 | Train Loss: 0.6431 | Val Loss: 0.6348
10
Epoch 10/200 | Train Loss: 0.6377 | Val Loss: 0.6264
11
Epoch 11/200 | Train Loss: 0.6281 | Val Loss: 0.6242
12
Epoch 12/200 | Train Loss: 0.6180 | Val Loss: 0.6035
13
Epoch 13/200 | Train Loss: 0.6060 | Val Loss: 0.5977
14
Epoch 14/200 | Train Loss: 0.6004 | Val Loss: 0.5893
15
Epoch 15/200 | Train Loss: 0.5948 | Val Loss: 0.5872
16
Epoch 16/200 | Train Loss: 0.5936 | Val Loss: 0.5929
17
Epoch 17/200 | Train Loss: 0.5890 | Val Loss: 0.5821
18
Epoch 18/200 | Train Loss: 0.5853 | Val Loss: 0.5815
19
Ep

In [27]:
# def load_graph2(graph_path):
#     G = nx.read_graphml(graph_path)
#     mapping = {nid: i for i, nid in enumerate(G.nodes())}
#     inv_map = {i: nid for nid, i in mapping.items()}
#     return G, mapping, inv_map

# # ====== SET YOUR FILE PATHS HERE ======
# graph_path = '/kaggle/input/final-graph/final_citations_graph.graphml'
# # emb_path = '/kaggle/input/embeddings/node_embeddings.pt'
# emb_path = '/kaggle/working/node_embeddings.pt'
# # model_path = '/kaggle/input/linkpred/pytorch/default/1/linkpred.pt'
# model_path = '/kaggle/working/linkpred.pt'
# k = 5
# # ====== LOAD DATA ======
# G, mapping, inv_map = load_graph2(graph_path)
# emb = torch.load(emb_path)
# # state = torch.load(model_path, map_location='cpu')
# # lin_weight = state['lin.weight']
# # lin_bias = state['lin.bias']

# # ====== INFERENCE LOOP ======
# test_nodes = [nid for nid in G.nodes() if G.out_degree(nid) > 0]

# print(len(test_nodes))
# cnt = 0
# for nid in test_nodes:
#     # u = mapping[nid]  # Get the index of the current node
#     # h_u = emb[u].unsqueeze(0)  # Embedding of node u, shape: (1, embedding_dim)
    
#     # # Calculate cosine similarity between h_u and all node embeddings
#     # cos_sim = F.cosine_similarity(h_u, emb, dim=1)
    
#     # # Set the score for the node itself (u) to a very low value (to avoid selecting the node itself)
#     # cos_sim[u] = -float('inf')
    
#     # # Get the top-k most similar nodes
#     # topk_scores, topk_indices = torch.topk(cos_sim, k)  # Get top-5 most similar nodes
    
#     # # Convert indices back to node IDs (if necessary)
#     # preds = [inv_map[idx] for idx in topk_indices.tolist()]
#     u = mapping[nid]  # Get the index of the current node
#     h_u = emb[u].unsqueeze(0)  # Shape: (1, embedding_dim)
    
#     # Compute dot product with all node embeddings
#     dot_sim = torch.matmul(emb, h_u.T).squeeze(1)  # Shape: (num_nodes,)
    
#     # Set self-score to -inf to avoid selecting the node itself
#     dot_sim[u] = -float('inf')
    
#     # Get the top-k most similar nodes
#     topk_scores, topk_indices = torch.topk(dot_sim, k)
    
#     # Convert indices back to node IDs
#     preds = [inv_map[idx] for idx in topk_indices.tolist()]

    
#     # Actual neighbors in the graph
#     actual = set(G.successors(nid))
    
#     # Evaluation metrics
#     hit = any(p in actual for p in preds)
#     correct = sum(1 for p in preds if p in actual)
#     precision = correct / len(preds) if preds else 0
#     recall = correct / len(actual) if actual else 0
    
#     # # Print results
#     # print(f'Node {nid}: Predicted={preds}, Actual={list(actual)}, Hit={hit}')
#     # print(f'Precision={precision:.2f}, Recall={recall:.2f}')
#     # print("==============================================")
#     if(hit):
#         cnt += 1
# print(cnt/6003)

# # for nid in test_nodes[30:40]:
# #     u = mapping[nid]
# #     h_u = emb[u].unsqueeze(0)
# #     h_u_rep = h_u.repeat(emb.size(0), 1)
# #     pair = torch.cat([h_u_rep, emb], dim=1)
# #     print(pair)
# #     scores = (pair @ lin_weight.t()).view(-1) + lin_bias
# #     scores[u] = -float('inf')
# #     topk = torch.topk(scores, k).indices.tolist()

# #     preds = [inv_map[idx] for idx in topk]
# #     actual = set(G.successors(nid))
# #     hit = any(p in actual for p in preds)
# #     correct = sum(1 for p in preds if p in actual)
# #     precision = correct / len(preds) if preds else 0
# #     recall = correct / len(actual) if actual else 0
# #     print(f'Node {nid}: Predicted={preds}, Actual={list(actual)}, Hit={hit}')
# #     print(f'Precision={precision:.2f}, Recall={recall:.2f}')
# #     print("==============================================")

/tmp/ipykernel_31/2849003629.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load(emb_path)


6003
0.4331167749458604


In [ ]:
# print(G.number_of_edges(), G.number_of_nodes())

In [ ]:
# print(state.keys())

In [27]:
!pip install torch-geometric
!pip install rapidfuzz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.6 MB/s eta 0:00:00a 0:00:01


In [49]:
import argparse
import networkx as nx
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import random # Added for shuffling nodes

import torch
import torch.nn.functional as F
from torch_geometric.nn import GraphSAGE
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling, to_networkx, subgraph # Added subgraph
from torch_geometric.loader import NeighborLoader

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

import os
from tqdm import tqdm
import re
from rapidfuzz import process, fuzz

# --- GRAPH BUILDER CODE (Keep as is) ---
def extract_brace_content(text, start_idx):
    # ... (keep original function)
    assert text[start_idx] == '{'
    depth = 0
    i = start_idx
    while i < len(text):
        if text[i] == '{':
            depth += 1
        elif text[i] == '}':
            depth -= 1
            if depth == 0:
                return text[start_idx+1:i], i + 1
        i += 1
    return "", i

def parse_bbl(bbl_path='t.bbl'):
    # ... (keep original function)
    try:
        with open(bbl_path, 'r', encoding='utf8') as f:
            text = f.read()
    except UnicodeDecodeError:
        with open(bbl_path, 'r', encoding='latin-1') as f:
            text = f.read()

    titles = []

    # 1. BibLaTeX \entry{...} blocks
    entry_blocks = re.findall(r'\\entry\{.*?\}\{.*?\}\{.*?\}(.*?)\\endentry', text, flags=re.DOTALL)
    for block in entry_blocks:
        idx = block.find(r'\field{title}')
        if idx != -1:
            brace_start = block.find('{', idx + len(r'\field{title}'))
            if brace_start != -1:
                raw_title, _ = extract_brace_content(block, brace_start)
                title = clean_title(raw_title)
                titles.append(title)

    # 2. \bib{key}{type}{...} blocks
    bib_blocks = re.findall(r'\\bib\{.*?\}\{.*?\}\{(.*?)\n\}', text, flags=re.DOTALL)
    for block in bib_blocks:
        match = re.search(r'title\s*=\s*\{(.*?)\}', block, flags=re.DOTALL)
        if match:
            title = clean_title(match.group(1))
            titles.append(title)

    # 3. \bibitem blocks
    bibitems = re.split(r"\\bibitem(?:\[[^\]]*\]){0,3}\{.*?\}", text)[1:]
    for block in bibitems:
        title = None

        # (a) \showarticletitle{...}
        match = re.search(r'\\showarticletitle\s*\{(.*?)\}', block, flags=re.DOTALL)
        if match:
            title = clean_title(match.group(1))
        else:
            # (b) \newblock ...
            parts = re.split(r"\\newblock", block)
            if len(parts) >= 2:
                title = clean_title(parts[1])
            else:
                # (c) ``...'' quoted title
                quote_match = re.search(r"``(.*?)''", block, flags=re.DOTALL)
                if quote_match:
                    title = clean_title(quote_match.group(1))
                else:
                    # (d) : {Title}
                    colon_title = re.search(r':[ \n]*\{(.*?)\}', block, flags=re.DOTALL)
                    if colon_title:
                        title = clean_title(colon_title.group(1))

        if title:
            titles.append(title)

    # 4. Handle cases like \url{...}
    urls = re.findall(r'\\url\{(.*?)\}', text)
    for url in urls:
        # Optionally, save URLs if needed
        pass
    # if len(titles)==0:
    #     print(bbl_path)
    return titles


def parse_bib(bib_file_path):
    # ... (keep original function)
    try:
        with open(bib_file_path, 'r', encoding='utf8') as f:
            bib_string = f.read()
    except UnicodeDecodeError:
        with open(bib_file_path, 'r', encoding='latin-1') as f:
            bib_string = f.read()

    pattern = r'(?:^|\n|,\s*)title\s*=\s*(?:\{|")(.*?)(?:\}|")'
    matches = re.findall(pattern, bib_string, flags=re.IGNORECASE | re.DOTALL)

    cleaned_titles = []
    for title in matches:
        title = title.strip()
        cleaned_titles.append(re.sub(r"^[\{\}]+|[\{\}]+$", "", title).strip())

    return cleaned_titles


def clean_title(raw):
    # ... (keep original function)
    raw = raw.replace('\n', ' ')
    raw = re.sub(r'\s+', ' ', raw)
    raw = re.sub(r'\{\\[a-zA-Z]+\s+(.*?)\}', r'\1', raw)  # {\em foo} → foo
    raw = re.sub(r'\\[a-zA-Z]+\s*', '', raw)              # \em, \url, etc
    raw = re.sub(r'[{}]', '', raw)                       # remove braces
    return raw.strip(' .')


def build_title_map(data_dir):
    # ... (keep original function)
    title_map = {}
    cnt = 0
    all_files = os.listdir(data_dir)
    print(f"Total items in data_dir: {len(all_files)}")
    for paper in tqdm(all_files, desc="Building title map"):
        pap_dir = os.path.join(data_dir, paper)
        tfile = os.path.join(pap_dir, 'title.txt')
        # Ensure it's a directory corresponding to a paper ID format if needed
        # Example check: if os.path.isdir(pap_dir) and re.match(r'^\d+$', paper):
        if os.path.isdir(pap_dir) and os.path.isfile(tfile):
            try:
                with open(tfile, 'r', encoding='utf8') as f:
                    title_map[paper] = f.read().strip()
            except Exception as e:
                print(f"Warning: Could not read title for {paper}: {e}")
        # else:
        #     print(f"Skipping {paper}, not a valid paper directory or title.txt missing.") # Optional debug
    print(f"Found {len(title_map)} papers with titles.")
    return title_map

def build_references(data_dir, paper_ids):
    # ... (Modifications: only process known paper_ids)
    """Map paper_id -> list of reference titles for given paper_ids."""
    refs = {}
    # print(f"Building references for {len(paper_ids)} papers.")
    for paper in tqdm(paper_ids, desc="Building references"): # Iterate through provided paper IDs
        pap_dir = os.path.join(data_dir, paper)
        if not os.path.isdir(pap_dir):
            # print(f"Directory not found for paper {paper}, skipping.") # Optional debug
            continue

        found_ref_file = False
        # Prioritize .bbl
        for fname in os.listdir(pap_dir):
            if fname.endswith('.bbl'):
                bbl_path = os.path.join(pap_dir, fname)
                try:
                    refs[paper] = parse_bbl(bbl_path)
                    found_ref_file = True
                    # print(f"Found {len(refs[paper])} refs in {fname} for {paper}") # Optional debug
                    break
                except Exception as e:
                    print(f"Warning: Could not parse {bbl_path}: {e}")
                    refs[paper] = [] # Assign empty list on error

        # Fallback to .bib if no .bbl found or parsed successfully
        if not found_ref_file:
            for fname in os.listdir(pap_dir):
                if fname.endswith('.bib'):
                    bib_path = os.path.join(pap_dir, fname)
                    try:
                        refs[paper] = parse_bib(bib_path)
                        found_ref_file = True
                        # print(f"Found {len(refs[paper])} refs in {fname} for {paper}") # Optional debug
                        break
                    except Exception as e:
                        print(f"Warning: Could not parse {bib_path}: {e}")
                        refs[paper] = [] # Assign empty list on error

        if not found_ref_file:
             # print(f"No .bbl or .bib found for {paper}") # Optional debug
             refs[paper] = [] # Ensure entry exists even if no refs found

    print(f"Built reference lists for {len(refs)} papers.")
    return refs


def match_refs_to_papers(refs, title_map, threshold=80):
    """Fuzzy match reference titles to known paper titles."""
    graph = nx.DiGraph()
    known_pids = list(title_map.keys()) # Nodes are based on title_map keys

    # Add nodes that have titles
    for pid in known_pids:
        graph.add_node(pid)

    titles = list(title_map.values())
    # Create reverse mapping only once
    pid_by_title = {v: k for k, v in title_map.items()}

    print(f"Matching references for {len(refs)} source papers against {len(titles)} target titles.")
    for src, rlist in tqdm(refs.items(), desc='Matching refs'):
        if src not in graph: # Ensure source node exists in our known set
             # print(f"Warning: Source paper {src} not in title_map, skipping its references.") # Optional Debug
             continue
        if not rlist: # Skip if no references were parsed for this paper
            continue

        # --- Loop through each reference title for the current source paper ---
        for ref in rlist:
            # Add a basic check for valid reference string to avoid errors with process.extract
            if not ref or not isinstance(ref, str) or len(ref.strip()) < 5: # Skip empty, non-string, or very short refs
                continue

            # --- Perform fuzzy matching for the *current* reference 'ref' ---
            # Option 1: Using extract (finds multiple matches above threshold)
            extracted_matches = process.extract(ref, titles, scorer=fuzz.token_set_ratio, score_cutoff=threshold)

            # Process the matches found for this specific 'ref'
            for match_title, score, _ in extracted_matches:
                tgt = pid_by_title.get(match_title) # Look up PID from matched title
                # Ensure target node exists in our graph nodes AND is not the source itself (optional)
                if tgt and tgt in graph:
                    # Optional: Prevent self-citations if desired
                    # if src == tgt:
                    #     continue
                    # Add edge only if it doesn't exist to avoid parallel edges if multiple refs match same target
                    if not graph.has_edge(src, tgt):
                        graph.add_edge(src, tgt)
                        # If you only want the *best* match per reference, you could break here
                        # break # Uncomment if you only want the first match above threshold per 'ref'


            # --- Option 2: Alternative using extractOne (finds only the single best match >= threshold) ---
            # Uncomment this block and comment out the 'Option 1' block above if you prefer extractOne
            # match_result = process.extractOne(ref, titles, scorer=fuzz.token_set_ratio, score_cutoff=threshold) # Use score_cutoff
            # if match_result: # Check if a match was found above the threshold
            #     match_title, score, _ = match_result
            #     tgt = pid_by_title.get(match_title)
            #     if tgt and tgt in graph:
            #         # Optional: Prevent self-citations
            #         # if src == tgt:
            #         #     continue
            #         if not graph.has_edge(src, tgt):
            #             graph.add_edge(src, tgt)
            # --- End Alternative ---

    print(f"Graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    return graph

# --- INVOCATION OF GRAPH BUILDER ---
def generate_graphml(data_dir, out_path):
    print("Building title map...")
    title_map = build_title_map(data_dir)
    if not title_map:
        print("Error: No titles found. Cannot build graph.")
        return None

    print("Building references...")
    # Pass only the paper IDs that have titles
    refs = build_references(data_dir, list(title_map.keys()))

    print("Matching references to papers...")
    G = match_refs_to_papers(refs, title_map)

    print(f"Writing graph to {out_path}...")
    nx.write_graphml(G, out_path)
    print(f"Graph stats: Nodes={G.number_of_nodes()}, Edges={G.number_of_edges()}")
    return G

# --- MODEL AND TRAINING ---

# 1. Prepare text embeddings (keep SentenceTransformer initialization global if needed)
# Consider moving initialization inside relevant functions if memory is tight
# sbert = SentenceTransformer('all-MiniLM-L6-v2') # Keep commented if loaded later

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_dim=384, hidden=128, num_layers=2):
        super().__init__()
        self.sage = GraphSAGE(in_channels=in_dim, hidden_channels=hidden, num_layers=num_layers, out_channels=hidden) # Explicitly set out_channels
        # No linear layer needed for dot product score

    def forward(self, x, edge_index):
         # Only encode nodes using GraphSAGE
        h = self.sage(x, edge_index)
        return h # Return node embeddings

    def decode(self, h, edge_label_index):
        # Decode links using dot product
        return (h[edge_label_index[0]] * h[edge_label_index[1]]).sum(dim=1)


# --- MODIFIED GRAPH LOADING ---
def load_full_graph_data(graph_path):
    """Loads the full graph and creates mappings."""
    print(f"Loading full graph from {graph_path}...")
    if not os.path.exists(graph_path):
         raise FileNotFoundError(f"Graph file not found: {graph_path}")
    Gnx = nx.read_graphml(graph_path)
    print(f"Full graph loaded: {Gnx.number_of_nodes()} nodes, {Gnx.number_of_edges()} edges.")

    nodes = list(Gnx.nodes())
    mapping = {nid: i for i, nid in enumerate(nodes)}
    inv_mapping = {i: nid for nid, i in mapping.items()}

    edges = [[mapping[u], mapping[v]] for u, v in Gnx.edges() if u in mapping and v in mapping]
    if not edges:
        print("Warning: No edges found or mapped in the graph!")
        edge_index = torch.empty((2, 0), dtype=torch.long)
    else:
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

    print(f"Full edge_index created with shape: {edge_index.shape}")
    return Gnx, edge_index, mapping, inv_mapping


# --- TEXT EMBEDDING LOADING ---
def load_text_embeddings(data_dir, node_list, mapping, device):
    """Loads or computes text embeddings for a given list of nodes."""
    # Ensure mapping covers all nodes in node_list
    if not all(nid in mapping for nid in node_list):
         missing = [nid for nid in node_list if nid not in mapping]
         print(f"Warning: {len(missing)} nodes in node_list are not in the provided mapping. This might cause index errors.")
         # Option 1: Filter node_list (if subsequent code handles potential missing nodes)
         # node_list = [nid for nid in node_list if nid in mapping]
         # Option 2: Raise an error
         raise ValueError(f"Mapping is incomplete for node_list. Missing: {missing[:5]}...")


    sbert = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    emb_dim = 384 # SBERT embedding dimension
    # Initialize embeddings for all nodes covered by the mapping
    x = torch.zeros((len(mapping), emb_dim), dtype=torch.float, device=device)
    print(f"Initialized embedding tensor shape: {x.shape}")

    processed_count = 0
    for nid in tqdm(node_list, desc='Embedding texts'):
        if nid not in mapping:
             # print(f"Skipping node {nid} as it's not in the mapping.") # Already warned above
             continue

        idx = mapping[nid]
        txt = ''
        nid_path = os.path.join(data_dir, str(nid)) # Ensure nid is string for path join
        # print(f"Processing node {nid} (index {idx}), path: {nid_path}") # Debug

        for fname in ('title.txt', 'abstract.txt'):
            path = os.path.join(nid_path, fname)
            if os.path.isfile(path):
                try:
                    with open(path, 'r', encoding='utf8') as f:
                        txt += f.read().strip() + ' '
                except Exception as e:
                    print(f"Warning: Could not read {path}: {e}")

        if txt.strip():
            try:
                # Ensure input is not excessively long (optional safeguard)
                max_len = 512 # Example limit, adjust as needed
                if len(txt.split()) > max_len:
                     txt = ' '.join(txt.split()[:max_len])

                emb = sbert.encode(txt.strip(), convert_to_tensor=True, show_progress_bar=False)
                # print(f"Node {nid}: Text length={len(txt.strip())}, Emb shape={emb.shape}") # Debug
                if emb.shape[0] == emb_dim:
                    x[idx] = emb.to(device) # Ensure tensor is on the correct device
                    processed_count += 1
                else:
                    print(f"Warning: Embedding for node {nid} has unexpected shape {emb.shape}. Skipping.")
            except Exception as e:
                print(f"Error encoding text for node {nid}: {e}")
                # Optionally handle error, e.g., skip node or use zero vector
        # else:
            # print(f"Node {nid}: No text found in title.txt or abstract.txt.") # Debug

    print(f"Successfully processed and embedded text for {processed_count} nodes out of {len(node_list)} requested.")

    # Normalize embeddings
    x = F.normalize(x, p=2, dim=1)
    # Replace NaNs resulting from normalizing zero vectors (if any)
    x = torch.nan_to_num(x)
    print("Embeddings normalized.")

    return x


# --- MODIFIED TRAINING FUNCTION ---
def train(args, x_train, edge_index_train, train_nodes_indices, device):
    print(f"Starting training on {device}...")
    print(f"Training data: Nodes={x_train.size(0)}, Edges={edge_index_train.size(1)}")

    # Ensure data is on the correct device
    x_train = x_train.to(device)
    edge_index_train = edge_index_train.to(device)

    # Split edges *within the training graph* for training and validation
    num_train_edges = edge_index_train.size(1)
    if num_train_edges == 0:
        print("Error: No edges in the training graph. Cannot train.")
        return None, None

    perm = torch.randperm(num_train_edges, device=device) # Generate permutation on the correct device
    val_cnt = int(args.val_ratio * num_train_edges)
    val_idx = perm[:val_cnt]
    train_idx = perm[val_cnt:]

    pos_train_edges = edge_index_train[:, train_idx]
    pos_val_edges = edge_index_train[:, val_idx]

    print(f"Positive edges split: Train={pos_train_edges.size(1)}, Val={pos_val_edges.size(1)}")

    # --- Create a reduced edge_index for message passing during training ---
    # This index should *only* contain the positive training edges.
    # The validation edges are held out and not used for message passing.
    train_msg_passing_edge_index = pos_train_edges

    model = LinkPredictor(in_dim=x_train.size(1), hidden=args.hidden_dim, num_layers=args.num_layers).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    print("Model and optimizer initialized.")

    best_val_loss = float('inf')
    epochs_no_improve = 0
    patience = 50 # Stop training if validation loss doesn't improve for 'patience' epochs

    for epoch in range(1, args.epochs + 1):
        model.train()
        optimizer.zero_grad()

        # Use only training edges for message passing
        h_train = model(x_train, train_msg_passing_edge_index)

        # Sample negative edges for training *within the training graph*
        # Ensure negative sampling operates correctly on the number of training nodes
        num_train_nodes = x_train.size(0)
        neg_train_edges = negative_sampling(
            edge_index=edge_index_train, # Sample based on all edges in train graph to avoid sampling existing links
            num_nodes=num_train_nodes,
            num_neg_samples=pos_train_edges.size(1),
            method='sparse'
        ).to(device)

        # Decode positive and negative training edges
        pos_train_logits = model.decode(h_train, pos_train_edges)
        neg_train_logits = model.decode(h_train, neg_train_edges)

        # Calculate training loss
        pos_train_loss = F.binary_cross_entropy_with_logits(pos_train_logits, torch.ones_like(pos_train_logits))
        neg_train_loss = F.binary_cross_entropy_with_logits(neg_train_logits, torch.zeros_like(neg_train_logits))
        train_loss = (pos_train_loss + neg_train_loss) / 2

        train_loss.backward()
        # Optional: Gradient clipping
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # --- Validation ---
        model.eval()
        with torch.no_grad():
            # Note: For validation, we still use the embeddings (h_train) generated
            # using ONLY the training message passing edges. We are evaluating how well
            # these embeddings predict the held-out validation links.
            h_val = h_train # Use the same embeddings

            # Sample negative edges for validation
            neg_val_edges = negative_sampling(
                edge_index=edge_index_train,
                num_nodes=num_train_nodes,
                num_neg_samples=pos_val_edges.size(1),
                method='sparse'
            ).to(device)

            # Decode positive and negative validation edges
            pos_val_logits = model.decode(h_val, pos_val_edges)
            neg_val_logits = model.decode(h_val, neg_val_edges)

            # Calculate validation loss
            pos_val_loss = F.binary_cross_entropy_with_logits(pos_val_logits, torch.ones_like(pos_val_logits))
            neg_val_loss = F.binary_cross_entropy_with_logits(neg_val_logits, torch.zeros_like(neg_val_logits))
            val_loss = (pos_val_loss + neg_val_loss) / 2

        print(f'Epoch {epoch}/{args.epochs} | Train Loss: {train_loss.item():.4f} | Val Loss: {val_loss.item():.4f}')

        # --- Early Stopping ---
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            # Save the best model state
            torch.save(model.state_dict(), args.out_model_path)
            print(f"Validation loss improved. Saved model to {args.out_model_path}")
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Validation loss did not improve for {patience} epochs. Early stopping.")
            break

    # --- Post-Training ---
    # Load the best model state
    print(f"Loading best model state from {args.out_model_path}")
    model.load_state_dict(torch.load(args.out_model_path, map_location=device))
    model.eval()

    # Generate final embeddings for the training nodes using the best model
    # Use the full training edge index (pos_train + pos_val) for generating final embeddings
    print("Generating final embeddings for training nodes using the best model...")
    with torch.no_grad():
        final_emb_train = model(x_train, edge_index_train).cpu() # Move to CPU for saving/analysis

    print(f"Final training node embeddings shape: {final_emb_train.shape}")

    # --- Sanity check embeddings (optional) ---
    # with torch.no_grad():
    #     per_node_std = final_emb_train.std(dim=1)
    #     per_node_mean = final_emb_train.mean(dim=1)
    #     print("Embedding stats for training nodes:")
    #     # print(f"Std[:10]: {per_node_std[:10]}")
    #     # print(f"Mean[:10]: {per_node_mean[:10]}")
    #     print(f"Mean of per-node std: {per_node_std.mean().item():.6f}")
    #     print(f"Mean of per-node mean: {per_node_mean.mean().item():.6f}")
    #     print(f"NaN count in embeddings: {torch.isnan(final_emb_train).sum().item()}")


    print('Training complete.')
    # The model state is already saved (best validation score)
    # Return the model (in eval mode) and the final training embeddings
    return model, final_emb_train


# --- Configuration Class ---
class Config:
    # --- Paths ---
    data_dir = '/kaggle/input/papers/dataset_papers' # Source data directory
    graph_dir = '/kaggle/working/' # Directory to save/load the graphml
    output_dir = '/kaggle/working/' # Directory for model outputs
    graph_name = 'citation_graph.graphml'
    model_name = 'linkpred_model_final.pt'
    embeddings_name = 'node_embeddings_final.pt' # For all nodes after inference
    text_embeddings_cache = '/kaggle/input/text-embeddings/text_embeddings.pt' # Cache for initial text embeds

    # --- Graph Generation ---
    generate_graph = False # Set to False if graphml already exists

    # --- Data Split ---
    test_node_ratio = 0.05 # Hold out 5% of nodes for testing
    val_ratio = 0.05 # Use 10% of *training graph edges* for validation during training

    # --- Model & Training ---
    hidden_dim = 64
    num_layers = 3
    epochs = 1000 # Reduced epochs for potentially faster convergence with early stopping
    lr = 2e-2 # Adjusted learning rate
    device = 'cpu' if torch.cuda.is_available() else 'cpu'
    weight_decay = 1e-4
    # --- Inference ---
    k = 10 # Top-k predictions for evaluation

    # --- Derived Paths ---
    @property
    def graph_path(self):
        if not self.generate_graph:
            return '/kaggle/input/final-graph/final_citations_graph.graphml'
        return os.path.join(self.graph_dir, self.graph_name)

    @property
    def out_model_path(self):
        return os.path.join(self.output_dir, self.model_name)

    @property
    def out_embeddings_path(self):
        return os.path.join(self.output_dir, self.embeddings_name)

# --- Main Workflow ---
def main_workflow():
    args = Config()
    device = torch.device(args.device)
    print(f"Using device: {device}")
    os.makedirs(args.graph_dir, exist_ok=True)
    os.makedirs(args.output_dir, exist_ok=True)

    # 1. Generate or Load Full Graph
    if args.generate_graph or not os.path.exists(args.graph_path):
        print("Generating GraphML file...")
        G_full = generate_graphml(args.data_dir, args.graph_path)
        if G_full is None:
            print("Graph generation failed. Exiting.")
            return
    else:
        print(f"Loading existing graph from {args.graph_path}")
        G_full = nx.read_graphml(args.graph_path)

    if G_full.number_of_nodes() == 0:
         print("Error: Loaded graph has no nodes. Check graph generation/loading.")
         return

    # 2. Load Full Graph Data (Edges, Mappings)
    G_full, edge_index_full, mapping_full, inv_map_full = load_full_graph_data(args.graph_path)
    all_node_ids = list(G_full.nodes())
    num_all_nodes = len(all_node_ids)
    print(f"Total nodes in graph: {num_all_nodes}")
    if num_all_nodes == 0:
        print("Error: No nodes found in the loaded graph.")
        return

    # 3. Load or Generate Initial Text Embeddings (for ALL nodes)
    if os.path.exists(args.text_embeddings_cache):
        print(f"Loading cached text embeddings from {args.text_embeddings_cache}")
        x_full = torch.load(args.text_embeddings_cache, map_location='cpu').to(device) # Load to CPU first, then move
        # Verify shape consistency
        if x_full.shape[0] != num_all_nodes:
             print(f"Warning: Cached embeddings shape ({x_full.shape[0]}) doesn't match number of nodes ({num_all_nodes}). Recomputing.")
             x_full = load_text_embeddings(args.data_dir, all_node_ids, mapping_full, device)
             torch.save(x_full.cpu(), args.text_embeddings_cache) # Save the recomputed embeddings
        else:
             print("Cached text embeddings loaded successfully.")

    else:
        print("Generating initial text embeddings for all nodes...")
        x_full = load_text_embeddings(args.data_dir, all_node_ids, mapping_full, device)
        print(f"Saving text embeddings to cache: {args.text_embeddings_cache}")
        torch.save(x_full.cpu(), args.text_embeddings_cache) # Save to CPU

    if x_full is None or x_full.shape[0] != num_all_nodes:
        print("Error: Failed to load or generate initial text embeddings correctly.")
        return
    print(f"Full text embeddings shape: {x_full.shape}")


    # 4. Split Nodes into Train and Test Sets
    print(f"Splitting nodes into training and testing sets ({1-args.test_node_ratio:.0%} train, {args.test_node_ratio:.0%} test)...")
    if len(all_node_ids) < 2: # Need at least 2 nodes for train/test split
        print("Error: Not enough nodes to perform train/test split.")
        return

    try:
        train_node_ids, test_node_ids = train_test_split(
            all_node_ids,
            test_size=args.test_node_ratio,
            random_state=42 # for reproducibility
        )
    except ValueError as e:
        print(f"Error during train/test split: {e}. Check node list and test_size.")
        # Fallback: If test_size is too small or causes issues with very few nodes
        if len(all_node_ids) > 0:
             print("Fallback: Assigning all nodes to training due to split error.")
             train_node_ids = all_node_ids
             test_node_ids = []
        else:
             print("Cannot proceed with zero nodes.")
             return


    num_train_nodes = len(train_node_ids)
    num_test_nodes = len(test_node_ids)
    print(f"Training nodes: {num_train_nodes}, Testing nodes: {num_test_nodes}")

    if num_train_nodes == 0:
        print("Error: No nodes assigned to the training set. Cannot train.")
        return

    # 5. Prepare Data for Training
    # Get indices of training nodes in the full mapping
    train_nodes_indices_full = torch.tensor([mapping_full[nid] for nid in train_node_ids], dtype=torch.long).to(device)

    # Select corresponding initial embeddings for training nodes
    x_train = x_full[train_nodes_indices_full]
    print(f"Training embeddings shape: {x_train.shape}")


    # Create the training edge_index (only edges *between* training nodes)
    # Use PyG's subgraph utility
    # device = torch.device(args.device)
    # print(f"Moving tensors to device: {device}")
    
    # # Assuming these tensors are created from your graph processing
    # edge_index_full = edge_index_full.to(device)
    # # If you have node features, move them too:
    # # x_full = x_full.to(device)
    # # Move your node index tensors
    # train_nodes_indices_full = train_nodes_indices_full.to(device)
    # test_nodes_indices_full = test_nodes_indices_full.to(device)
    # # Move any other tensors that will be used in device-specific operations
    
    # # --- Now call subgraph with tensors on the correct device ---
    # # This part of your code remains the same, but the inputs are now on the right device
    # edge_index_train, train_subgraph_edge_attr = subgraph(
    #     subset=train_nodes_indices_full, # This is now on the device
    #     edge_index=edge_index_full,     # This is now on the device
    #     # edge_attr=edge_attr_full, # If you use edge attributes, move them too
    #     relabel_nodes=True # Or False depending on your needs
    #     # num_nodes=G_full.number_of_nodes() # Pass if relabeling
    # )

    edge_index_train, train_subgraph_edge_attr = subgraph(
        subset=train_nodes_indices_full,
        edge_index=edge_index_full,
        relabel_nodes=True, # IMPORTANT: Relabel node indices to be 0 to num_train_nodes-1
        num_nodes=num_all_nodes # Specify total number of nodes in original graph
    )
    # train_nodes_indices are now the relabeled indices (0 to N_train-1)
    # We need a mapping from the *new* 0-based indices back to the original *full* indices if needed later,
    # but the train function itself works with the relabeled indices and x_train directly.
    print(f"Training edge_index shape (relabeled): {edge_index_train.shape}")


    # 6. Train the Model
    model, _ = train(args, x_train, edge_index_train, torch.arange(num_train_nodes), device) # Pass relabeled indices 0..N_train-1

    if model is None:
        print("Model training failed.")
        return

    # 7. Inference: Generate Embeddings for ALL Nodes using the Trained Model
    # print("Performing inference to generate embeddings for ALL nodes...")
    # model.eval() # Ensure model is in evaluation mode
    # with torch.no_grad():
    #     # Use the trained SAGE layers (`model.sage`) on the *full* graph and *full* initial features
    #     # The `forward` method of LinkPredictor class IS the sage layer call
    #     final_emb_full = model(x_full, edge_index_full).cpu() # Move to CPU

    # print(f"Final embeddings generated for all nodes. Shape: {final_emb_full.shape}")
    # print(f"Saving final full embeddings to {args.out_embeddings_path}")
    # torch.save(final_emb_full, args.out_embeddings_path)
    test_idx = torch.tensor([mapping_full[nid] for nid in test_node_ids], dtype=torch.long, device=device)
    
    # Create a mask that keeps only edges where neither endpoint is a test‐node
    mask = ~(
        torch.isin(edge_index_full[0], test_idx) |
        torch.isin(edge_index_full[1], test_idx)
    )
    
    # Apply the mask to filter out any edge touching a test node
    edge_index_no_test = edge_index_full[:, mask]
    
    # Now use this filtered edge_index for inference
    print("Performing inference with test‐nodes isolated…")
    model.eval()
    with torch.no_grad():
        final_emb_full = model(x_full, edge_index_no_test).cpu()

    print(f"Final embeddings generated for all nodes. Shape: {final_emb_full.shape}")
    print(f"Saving final full embeddings to {args.out_embeddings_path}")
    torch.save(final_emb_full, args.out_embeddings_path)



    # 8. Evaluate on Test Nodes
    print(f"\n--- Evaluating on {num_test_nodes} Test Nodes ---")
    if num_test_nodes == 0:
        print("No test nodes to evaluate.")
        return

    # Use the final embeddings generated in step 7
    emb_eval = final_emb_full.to(device) # Move back to device if needed for calculations
    print(emb_eval.shape)
    total_hit = 0
    total_precision = 0
    total_recall = 0
    evaluated_nodes = 0

    for test_nid in tqdm(test_node_ids, desc="Evaluating test nodes"):
        if test_nid not in mapping_full:
            print(f"Warning: Test node {test_nid} not found in full mapping. Skipping.")
            continue

        u_idx_full = mapping_full[test_nid] # Index in the full graph/embedding tensor
        h_u = emb_eval[u_idx_full].unsqueeze(0) # Embedding of the test node

        # Calculate scores (dot product) between the test node and ALL other nodes
        # Ensure dimensions match for matmul: emb_eval (N, D), h_u.T (D, 1) -> scores (N, 1) -> squeeze -> (N,)
        scores = torch.matmul(emb_eval, h_u.T).squeeze(1)

        # Set score for the node itself to -inf to avoid self-prediction
        scores[u_idx_full] = -float('inf')

        # --- Optional: Exclude known training nodes from predictions ---
        # If you ONLY want to predict links to OTHER UNSEEN (test) nodes:
        # test_nodes_indices_full = torch.tensor([mapping_full[nid] for nid in test_node_ids], dtype=torch.long).to(device)
        # scores[train_nodes_indices_full] = -float('inf') # Exclude training nodes
        # --- Standard approach: Predict links to ANY node (train or test) ---
        pass # Keep predictions for all other nodes

        # Get top-k predictions based on scores
        try:
             # Ensure k is not larger than the number of possible prediction targets
            effective_k = min(args.k, emb_eval.size(0) - 1) # -1 for self-node
            if effective_k <=0:
                 print(f"Warning: Not enough other nodes to predict for node {test_nid}. Skipping k={args.k} prediction.")
                 continue # Skip if k becomes 0 or negative

            topk_scores, topk_indices_full = torch.topk(scores, effective_k)

            # Map predicted indices back to original node IDs
            pred_node_ids = [inv_map_full[idx.item()] for idx in topk_indices_full]

        except RuntimeError as e:
            print(f"Error during topk calculation for node {test_nid}: {e}. Scores shape: {scores.shape}, k={args.k}")
            continue # Skip this node if topk fails


        # Get actual neighbors (successors) from the original full graph G_full
        actual_neighbors = set(G_full.successors(test_nid))

        if not actual_neighbors: # Skip evaluation if the test node has no outgoing links in reality
            # print(f"Node {test_nid} has no actual successors. Skipping eval for this node.")
            continue

        # Calculate metrics
        pred_set = set(pred_node_ids)
        correct_preds = actual_neighbors.intersection(pred_set)

        hit = 1 if len(correct_preds) > 0 else 0
        precision = len(correct_preds) / len(pred_set) if pred_set else 0
        recall = len(correct_preds) / len(actual_neighbors) if actual_neighbors else 0

        total_hit += hit
        total_precision += precision
        total_recall += recall
        evaluated_nodes += 1

        # Optional: Print per-node results (can be verbose)
        # print(f'Node {test_nid}: Pred={pred_node_ids}, Actual={list(actual_neighbors)}, Hit={hit}, Prec={precision:.2f}, Rec={recall:.2f}')

    # Calculate average metrics
    avg_hit_rate = total_hit / evaluated_nodes if evaluated_nodes > 0 else 0
    avg_precision = total_precision / evaluated_nodes if evaluated_nodes > 0 else 0
    avg_recall = total_recall / evaluated_nodes if evaluated_nodes > 0 else 0

    print("\n--- Evaluation Summary ---")
    print(f"Evaluated on {evaluated_nodes} test nodes (nodes with actual outgoing links).")
    print(f"Average Hit Rate @{args.k}: {avg_hit_rate:.4f}")
    print(f"Average Precision @{args.k}: {avg_precision:.4f}")
    print(f"Average Recall @{args.k}: {avg_recall:.4f}")


if __name__ == "__main__":
    # Make sure to set the paths correctly in the Config class above
    # Especially args.data_dir
    main_workflow()

Using device: cpu
Loading existing graph from /kaggle/input/final-graph/final_citations_graph.graphml
Loading full graph from /kaggle/input/final-graph/final_citations_graph.graphml...
Full graph loaded: 6545 nodes, 28562 edges.
Full edge_index created with shape: torch.Size([2, 28562])
Total nodes in graph: 6545
Loading cached text embeddings from /kaggle/input/text-embeddings/text_embeddings.pt
Cached text embeddings loaded successfully.
Full text embeddings shape: torch.Size([6545, 384])
Splitting nodes into training and testing sets (95% train, 5% test)...
Training nodes: 6217, Testing nodes: 328
Training embeddings shape: torch.Size([6217, 384])
Training edge_index shape (relabeled): torch.Size([2, 25370])
Starting training on cpu...
Training data: Nodes=6217, Edges=25370
Positive edges split: Train=24102, Val=1268
Model and optimizer initialized.
Epoch 1/1000 | Train Loss: 0.7089 | Val Loss: 0.7087
Validation loss improved. Saved model to /kaggle/working/linkpred_model_final.pt


/tmp/ipykernel_31/2875912480.py:626: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_full = torch.load(args.text_embeddings_cache, map_location='cpu').to(device) # Load to C

Epoch 2/1000 | Train Loss: 1.3391 | Val Loss: 1.3357
Epoch 3/1000 | Train Loss: 0.6920 | Val Loss: 0.6926
Validation loss improved. Saved model to /kaggle/working/linkpred_model_final.pt
Epoch 4/1000 | Train Loss: 0.7119 | Val Loss: 0.7151
Epoch 5/1000 | Train Loss: 0.7000 | Val Loss: 0.7013
Epoch 6/1000 | Train Loss: 0.6938 | Val Loss: 0.6937
Epoch 7/1000 | Train Loss: 0.6724 | Val Loss: 0.6757
Validation loss improved. Saved model to /kaggle/working/linkpred_model_final.pt
Epoch 8/1000 | Train Loss: 0.8050 | Val Loss: 0.8044
Epoch 9/1000 | Train Loss: 0.6668 | Val Loss: 0.6721
Validation loss improved. Saved model to /kaggle/working/linkpred_model_final.pt
Epoch 10/1000 | Train Loss: 0.6865 | Val Loss: 0.6870
Epoch 11/1000 | Train Loss: 0.6975 | Val Loss: 0.6974
Epoch 12/1000 | Train Loss: 0.7011 | Val Loss: 0.7010
Epoch 13/1000 | Train Loss: 0.7027 | Val Loss: 0.7027
Epoch 14/1000 | Train Loss: 0.7021 | Val Loss: 0.7022
Epoch 15/1000 | Train Loss: 0.7002 | Val Loss: 0.6998
Epoch 16/

/tmp/ipykernel_31/2875912480.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.out_model_path, map_location=device))


Epoch 236/1000 | Train Loss: 0.5165 | Val Loss: 0.5379
Validation loss did not improve for 50 epochs. Early stopping.
Loading best model state from /kaggle/working/linkpred_model_final.pt
Generating final embeddings for training nodes using the best model...
Final training node embeddings shape: torch.Size([6217, 64])
Training complete.
Performing inference with test‐nodes isolated…
Final embeddings generated for all nodes. Shape: torch.Size([6545, 64])
Saving final full embeddings to /kaggle/working/node_embeddings_final.pt

--- Evaluating on 328 Test Nodes ---
torch.Size([6545, 64])


Evaluating test nodes: 100%|██████████| 328/328 [00:00<00:00, 5063.66it/s]


--- Evaluation Summary ---
Evaluated on 301 test nodes (nodes with actual outgoing links).
Average Hit Rate @10: 0.2159
Average Precision @10: 0.0272
Average Recall @10: 0.0485


In [4]:
title = 'Neural Tangent Kernel'
abstract = 'This shows theoritical approximations'

In [13]:
from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('all-MiniLM-L6-v2')

def convert_str_to_embeddding(txt):
    max_len = 512
    if len(txt.split()) > max_len:
        txt = ' '.join(txt.split()[:max_len])
    emb = sbert.encode(txt.strip(), convert_to_tensor=True, show_progress_bar=False)
    return emb.to('cpu')

emb = convert_str_to_embeddding(title+' '+abstract)

In [14]:
import torch
w = torch.load('/kaggle/input/text-embeddings/text_embeddings.pt')
w.shape

/tmp/ipykernel_31/3787199675.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  w = torch.load('/kaggle/input/text-embeddings/text_embeddings.pt')


torch.Size([6545, 384])

In [15]:
emb.device

device(type='cpu')

In [16]:
x_full = torch.cat([w, emb.unsqueeze(0)], dim=0)
x_full.shape

torch.Size([6546, 384])

In [19]:
import os
import networkx as nx
def load_full_graph_data(graph_path):
    if not os.path.exists(graph_path):
         raise FileNotFoundError(f"Graph file not found: {graph_path}")
    Gnx = nx.read_graphml(graph_path)
    print(f"Full graph loaded: {Gnx.number_of_nodes()} nodes, {Gnx.number_of_edges()} edges.")

    nodes = list(Gnx.nodes())
    mapping = {nid: i for i, nid in enumerate(nodes)}
    inv_mapping = {i: nid for nid, i in mapping.items()}

    edges = [[mapping[u], mapping[v]] for u, v in Gnx.edges() if u in mapping and v in mapping]
    if not edges:
        print("Warning: No edges found or mapped in the graph!")
        edge_index = torch.empty((2, 0), dtype=torch.long)
    else:
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

    print(f"Full edge_index created with shape: {edge_index.shape}")
    return Gnx, edge_index

G, edge_index = load_full_graph_data('/kaggle/input/final-graph/final_citations_graph.graphml')

Full graph loaded: 6545 nodes, 28562 edges.
Full edge_index created with shape: torch.Size([2, 28562])


In [23]:
!pip install torch_geometric

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00a 0:00:01


In [31]:
from torch_geometric.nn import GraphSAGE

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_dim=384, hidden=128, num_layers=2):
        super().__init__()
        self.sage = GraphSAGE(in_channels=in_dim, hidden_channels=hidden, num_layers=num_layers, out_channels=hidden) # Explicitly set out_channels
        # No linear layer needed for dot product score

    def forward(self, x, edge_index):
         # Only encode nodes using GraphSAGE
        h = self.sage(x, edge_index)
        return h # Return node embeddings

    def decode(self, h, edge_label_index):
        # Decode links using dot product
        return (h[edge_label_index[0]] * h[edge_label_index[1]]).sum(dim=1)
        
model = LinkPredictor(in_dim=x_full.size(1), hidden=64, num_layers=3).to('cpu')
model.load_state_dict(torch.load('/kaggle/working/linkpred_model_final.pt', map_location='cpu'))


/tmp/ipykernel_31/3942004862.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/linkpred_model_final.pt', map_location='c

<All keys matched successfully>

In [32]:
full_node_emb = model(x_full, edge_index).cpu()
full_node_emb.shape

torch.Size([6546, 64])

In [35]:
nodes = list(G.nodes())
mapping = {nid: i for i, nid in enumerate(nodes)}
inv_mapping = {i: nid for nid, i in mapping.items()}

In [37]:
query_emb = full_node_emb[-1]
scores = torch.matmul(full_node_emb, query_emb)
topk_scores, topk_indices_full = torch.topk(scores, k=10)
topk_indices_full

tensor([5190,  869, 1869, 3191, 3307, 1602, 1981, 5452,  313, 3536])

In [38]:
title_map = build_title_map('/kaggle/input/papers/dataset_papers')

Total items in data_dir: 6545


Building title map: 100%|██████████| 6545/6545 [01:22<00:00, 79.52it/s] 

Found 6545 papers with titles.


In [46]:
torch.save(title_map, 'title_map.pt')

In [47]:
title_map1 = torch.load('title_map.pt')

/tmp/ipykernel_31/3336012522.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  title_map1 = torch.load('title_map.pt')


In [48]:
title_map1 == title_map

True

In [44]:
for idx in topk_indices_full.tolist():
    print(title_map[inv_mapping[idx]])

Approximate Inference Turns Deep Networks into Gaussian Processes
Bayesian GAN
Random Feature Expansions for Deep Gaussian Processes
A Rigorous Link between Deep Ensembles and (Variational) Bayesian Methods
Practical Deep Learning with Bayesian Principles
Fast and Scalable Bayesian Deep Learning by Weight-Perturbation in Adam
Global inducing point variational posteriors for Bayesian neural networks and deep Gaussian processes
Multiplicative Normalizing Flows for Variational Bayesian Neural Networks
Accelerated Linearized Laplace Approximation for Bayesian Deep Learning
Liberty or Depth: Deep Bayesian Neural Nets Do Not Need Complex Weight Posterior Approximations
